In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import re
import math
from random import randint
import os
from time import sleep

# Run Behind Proxies

In [5]:
phantom_path = '/usr/local/bin/phantomjs'

service_args = ['--proxy=proxy-nl.privateinternetaccess.com:1080', 
                '--proxy-type=socks5', 
                '--proxy-auth=x6122622:ZzQCgNVhum',
                '--ignore-ssl-errors=true']

driver = webdriver.PhantomJS(phantom_path, service_args=service_args)
driver.get('http://www.showmyip.gr/')

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

<span class="ip_address">109.201.154.230
</span>

In [6]:
# get tag with ip address
ip_tag = soup.find(attrs={'class': 'ip_address'})
ip = (re.findall(r'''([0-9.]+)''', str(ip_tag)))[0]
ip

'109.201.154.230'

import requests
import socket
import socks

PORT = 1080
# no authentication
# socks.set_default_proxy(socks.SOCKS5, "localhost")
# with Authentication
socks.set_default_proxy(socks.SOCKS5, "proxy-nl.privateinternetaccess.com",PORT,True,"x6122622","ZzQCgNVhum")
socket.socket = socks.socksocket

# Show Current IP

In [7]:
def print_ip():
    try:
        url = 'http://www.showmyip.gr/'
        driver.get(url)

        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # get tag with ip address
        ip_tag = soup.find(attrs={'class': 'ip_address'})
        ip = (re.findall(r'''([0-9.]+)''', str(ip_tag)))[0]
        return ip
    except:
        return '#'*30

In [8]:
print_ip()

'46.166.137.214'

# Get Number of Pages for Title

In [9]:
def num_results(title):
    url = 'http://www.indeed.com/resumes?q={0}&co=US'.format(title)
    driver.get(url)
    
    # get page with selenium and pass to beautiful soup
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # get number of results for query
    result_count = soup.find(attrs={'id': 'result_count'})

    try:
        # grab number only
        num = (re.findall(r'''([0-9,]+)''', str(result_count)))[0]
        num = int(num.replace(',', ''))
        return num
    except:
        return 0

In [10]:
num_results('engineer')

1763752

# Navigate to Subsequent Pages

In [11]:
def navigate(start_page):
# return web page source for scraping names
    try:
        next_button = driver.find_element_by_xpath('//a[@href="{0}"]'.format('?q=c&co=US&start={0}'.format(str(start_page))))
        next_button.click()
    except:
        pass
    
    #sleep(5)
    html = driver.page_source
    
    return BeautifulSoup(html, 'lxml')

In [16]:
#navigate(100)

In [13]:
def get_users(start_page):
    temp = []
    ct = 0
    print(len(temp))
    while len(temp) == 0:
        try:
            print('Trying to get users users - start page: {0}'.format(start_page))
            # attempt url retrieve
            try:
                #hf = '?q=c&co=US&start={0}'.format(str(start_page))
                #next_button = driver.find_element_by_xpath('//a[@href="'+variable+'"]')
                next_button = driver.findElement(By.linkText('2')).click()
                next_button.click()
                sleep(5)
            except:
                pass

            #sleep(5)
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            
            # find users
            list_of_people = soup.find_all(attrs={'class':'clickable_resume_card'})

            # make list of users found
            temp = [re.search(r'''(\/.+)\?''', person.get('onclick')).group() for person in list_of_people]
        except:
            ct += 1
            print('Trying to get users again: {0}'.format(ct))
            sleep(15)
            temp = []

    return temp

In [14]:
links = driver.find_elements_by_partial_link_text('')
for link in links:
    print(link.get_attribute("href"))

NoSuchElementException: Message: {"request":{"headers":{"Accept":"application/json","Accept-Encoding":"identity","Connection":"close","Content-Length":"96","Content-Type":"application/json;charset=UTF-8","Host":"127.0.0.1:57486","User-Agent":"Python-urllib/3.5"},"httpVersion":"1.1","method":"POST","post":"{\"using\": \"partial link text\", \"sessionId\": \"f9819500-602e-11e6-8194-0be9d4240766\", \"value\": \"\"}","url":"/elements","urlParsed":{"anchor":"","query":"","file":"elements","directory":"/","path":"/elements","relative":"/elements","port":"","host":"","password":"","user":"","userInfo":"","authority":"","protocol":"","source":"/elements","queryKey":{},"chunks":["elements"]},"urlOriginal":"/session/f9819500-602e-11e6-8194-0be9d4240766/elements"}}
Screenshot: available via screen


In [19]:
start_page = 100
hf = '?q=c&co=US&start={0}'.format(str(start_page))
next_button = driver.find_element_by_xpath('//href="'+hf+'"]')
#next_button = driver.findElement(By.linkText('2')).click()
next_button.click()

InvalidSelectorException: Message: {"errorMessage":"Unable to locate an element with the xpath expression //href=\"?q=c&co=US&start=100\"] because of the following error:\nError: INVALID_EXPRESSION_ERR: DOM XPath Exception 51","request":{"headers":{"Accept":"application/json","Accept-Encoding":"identity","Connection":"close","Content-Length":"116","Content-Type":"application/json;charset=UTF-8","Host":"127.0.0.1:57486","User-Agent":"Python-urllib/3.5"},"httpVersion":"1.1","method":"POST","post":"{\"using\": \"xpath\", \"sessionId\": \"f9819500-602e-11e6-8194-0be9d4240766\", \"value\": \"//href=\\\"?q=c&co=US&start=100\\\"]\"}","url":"/element","urlParsed":{"anchor":"","query":"","file":"element","directory":"/","path":"/element","relative":"/element","port":"","host":"","password":"","user":"","userInfo":"","authority":"","protocol":"","source":"/element","queryKey":{},"chunks":["element"]},"urlOriginal":"/session/f9819500-602e-11e6-8194-0be9d4240766/element"}}
Screenshot: available vi

In [18]:
get_users(150)

0
Trying to get users users - start page: 150


['/r/Petros-Gazazyan/2b5b06cff39ce808?',
 '/r/Travis-London/8fe4de80947b60f2?',
 '/r/Robert-Luenebrink/9c6eeb04e6963d6b?',
 '/r/Joel-Darr/acbcc02f2044b655?',
 '/r/Melvin-Williams/6d757143003b8a7b?',
 '/r/Robert-Musso/2cb4acba17794174?',
 '/r/Kentraye-Williams/f03c234aaff38997?',
 '/r/Omjee-Sharma/ddb616c36d532de3?',
 '/r/Saiquone-Selby/fcd4b21a0ad8e28c?',
 '/r/Stephen-A-Kraft---Mechanical-Engineer/1c009e8f7e2f5309?',
 '/r/Sunna-Kristinsdottir/b2dd44fb646b3c2c?',
 '/r/Peter-Link/beb0af9ae3c5a4ce?',
 '/r/Shade-Fulton/4feea7a69ab74b9e?',
 '/r/Christopher-Hudson/312348c3980a57af?',
 '/r/5b165ca433e5dffc?',
 '/r/Joshi-Sr-Devops-Engineer/0fcd150d56bf8f54?',
 '/r/Gary-Williams/b02082ffe8681726?',
 '/r/Kevin-Rogers/9741f95fbcd9fab7?',
 '/r/Julian-Ramirez-De-Arellano/952287aef14626c8?',
 '/me/stevesigg?',
 '/r/Service-Engineer-Karumuri-Anusha/70b98792c31a9b25?',
 '/r/Robert-Walker/53a9d581661d4560?',
 '/r/Sales-Engineer-Khan/d343fda31431adc9?',
 '/r/Josh-Sloan/43b8ce3756ee1533?',
 '/r/Michael-S

In [ ]:
def scrape_users(title, country):
    # get starting page
    #driver.get('http://www.indeed.com/resumes?q=engineer&co=US')

    # cleanup previous file versions before starting due to append
    f = '{0}.csv'.format(title)
    if os.path.exists(f):
        os.remove(f)
    
    f = 'failed.csv'
    if os.path.exists(f):
        os.remove(f)
    
    # loop through pages
    for start_page in range(50, num_results(title), 50):
            # play nice
            sleep(randint(1,10))
            print('Done sleeping - start page: {0}'.format(start_page))

            # make list of users found
            temp = navigate(start_page)
            
            # show what ip is being used
            print('List: {0} --- Start page: {1} --- IP: {2}'.format(len(temp), start_page, print_ip()))
            
            # write list of names found to file
            title = title.replace(' ', '_')
            with open('{0}.csv'.format(title), 'a') as outfile:
                for name in temp:
                    outfile.write('{0}, \n'.format(name))

In [ ]:
def failed_page(title, start_page):
    with open('failed.csv', 'a') as outfile:
        outfile.write(('title:{0}, page_start:{1}, \n').format(title, start_page))

In [ ]:
scrape_users('engineer', 'US')

In [ ]:

# Start socks5 proxies with these commands run in 
# separate tmux tabs. Do not quit the proxies while
# scraping. I've only used the server shortcut
# because I have them setup in my ssh config.
# However, you can use username@server instead.

# ssh -ND 8081 nestanmr
# ssh -ND 8082 backupsy
# ssh -ND 8083 aws
# ssh -ND 8084 vultr


# Install phantomjs 2 on ubuntu with these commands:
# sudo add-apt-repository ppa:tanguy-patte/phantomjs
# sudo apt-get update
# sudo apt-get install phatomjs

phantom_path = '/usr/bin/phantomjs'

# Start the browser instances
service_args1 = ['--proxy=127.0.0.1:8081', '--proxy-type=socks5']
service_args2 = ['--proxy=127.0.0.1:8082', '--proxy-type=socks5']
service_args3 = ['--proxy=127.0.0.1:8083', '--proxy-type=socks5']
service_args4 = ['--proxy=127.0.0.1:8084', '--proxy-type=socks5']

driver1 = webdriver.PhantomJS(phantom_path, service_args=service_args1)
driver2 = webdriver.PhantomJS(phantom_path, service_args=service_args2)
driver3 = webdriver.PhantomJS(phantom_path, service_args=service_args3)
driver4 = webdriver.PhantomJS(phantom_path, service_args=service_args4)

driver_list = [driver1, driver2, driver3, driver4]

# Open each instance and login (you may not have to do this)

for driver in driver_list:

    driver.get('https://www.winespectator.com/auth/login')

    userid = driver.find_element_by_name('userid')
    userid.send_keys(winespectator_login_name)

    passwd = driver.find_element_by_name('passwd')
    passwd.send_keys(winespectator_password)

    login = driver.find_element_by_id('target')
    login.click()

    time.sleep(1.5)


# Here is the function that operates on each URL I want to scrape

for url in enumerate(url_list):
    if (url[0] % 100) == 0:
        print url[0]
        
    # Randomly pick the driver
    driver = np.random.choice(driver_list)
    
    # Load the URL 
    full_url = 'http://www.winespectator.com/' + url[1]
    driver.get(full_url)
    time.sleep(1.5)
    
    # Get the HTML
    html = driver.page_source

    # .... scrape the wine info

